In [2]:
import pandas as pd 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate

In [3]:
data= pd.read_csv("D:\PFE\Fact_Table_Rev.csv", low_memory=False)

In [4]:
data

,montant_HT,montant_ttc,date_saisie,date_facture,Fk_prestation,Fk_famille_prestation,Fk_nationnalite,Fk_Societe,Fk_typesociete
0,4.0,3.571429,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
1,6.0,5.357143,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
2,8.0,7.142857,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
3,10.0,8.928571,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
4,8.5,7.589286,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
...,...,...,...,...,...,...,...,...,...
937562,10.5,9.375000,2013-09-24,2013-09-24,BOI,BAR,LI,GRINPARAPH,LAB
937563,7.0,6.250000,2013-09-24,2013-09-24,BOI,BAR,LI,GRINPARAPH,LAB
937564,3.5,3.125000,2013-09-24,2013-09-24,BOIR,CAVE,TN,STANDART,SOC
937565,196.0,175.000000,2013-09-24,2013-09-24,REP,RESTO,TN,STANDART,SOC


In [5]:
df = pd.DataFrame(data, columns=["montant_HT", "date_saisie", "Fk_famille_prestation","Fk_typesociete"])

In [6]:
df['date_saisie'] = pd.to_datetime(df['date_saisie'])

In [6]:
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete
0,4.0,2006-09-18,BAR,SOC
1,6.0,2006-09-18,BAR,SOC
2,8.0,2006-09-18,BAR,SOC
3,10.0,2006-09-18,BAR,SOC
4,8.5,2006-09-18,BAR,SOC
...,...,...,...,...
937562,10.5,2013-09-24,BAR,LAB
937563,7.0,2013-09-24,BAR,LAB
937564,3.5,2013-09-24,CAVE,SOC
937565,196.0,2013-09-24,RESTO,SOC


In [7]:
df= df.drop(index=df.loc[(df['date_saisie'] < '2006-01-01') | (df['date_saisie'] > '2019-12-31')].index)

In [8]:
label_encoder = LabelEncoder()
df['Fk_prestation_code'] = label_encoder.fit_transform(df['Fk_famille_prestation'])
label_encoder = LabelEncoder()
df['Fk_typesociete_code'] = label_encoder.fit_transform(df['Fk_typesociete'])
df.head()

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
0,4.0,2006-09-18,BAR,SOC,2,5
1,6.0,2006-09-18,BAR,SOC,2,5
2,8.0,2006-09-18,BAR,SOC,2,5
3,10.0,2006-09-18,BAR,SOC,2,5
4,8.5,2006-09-18,BAR,SOC,2,5


In [20]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [14]:
selected_columns = ['date_saisie', 'montant_HT','Fk_prestation_code']
df1 = df.loc[:, selected_columns]
df1['year'] = df['date_saisie'].dt.year.astype(int)
df1['month'] = df['date_saisie'].dt.month.astype(int)
df1['day'] = df['date_saisie'].dt.day.astype(int)
selected_columns = ['year','month', 'day','montant_HT','Fk_prestation_code']
df1 = df1.loc[:, selected_columns]
df1

,year,month,day,montant_HT,Fk_prestation_code
0,2006,9,18,4.0,2
1,2006,9,18,6.0,2
2,2006,9,18,8.0,2
3,2006,9,18,10.0,2
4,2006,9,18,8.5,2
...,...,...,...,...,...
937562,2013,9,24,10.5,2
937563,2013,9,24,7.0,2
937564,2013,9,24,3.5,3
937565,2013,9,24,196.0,15


In [15]:
train_data = df1[(df1['year'] <= 2018)]
test_data = df1[df1['year'] == 2019]

In [15]:

train_data

,year,month,day,montant_HT,Fk_prestation_code
0,2006,9,18,4.0,2
1,2006,9,18,6.0,2
2,2006,9,18,8.0,2
3,2006,9,18,10.0,2
4,2006,9,18,8.5,2
...,...,...,...,...,...
937562,2013,9,24,10.5,2
937563,2013,9,24,7.0,2
937564,2013,9,24,3.5,3
937565,2013,9,24,196.0,15


In [16]:
train_data = train_data.groupby([ 'year','month','day' ,'Fk_prestation_code'])['montant_HT'].sum().reset_index()
test_data  = test_data .groupby([ 'year','month','day', 'Fk_prestation_code'])['montant_HT'].sum().reset_index()

In [17]:
test_data

,year,month,day,Fk_prestation_code,montant_HT
0,2019,1,1,2,320.673
1,2019,1,1,3,10.000
2,2019,1,1,6,11.000
3,2019,1,1,9,4.673
4,2019,1,1,14,185.907
...,...,...,...,...,...
3143,2019,12,31,12,15.000
3144,2019,12,31,14,548.000
3145,2019,12,31,15,9630.500
3146,2019,12,31,17,2790.400


In [50]:
test_data.dtypes

year                    int32
month                   int32
day                     int32
Fk_prestation_code      int32
montant_HT            float64
dtype: object

In [35]:
train_data_array = train_data.values
test_data_array = test_data.values

In [36]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train_data = scaler.fit_transform(train_data_array)
scaled_test_data = scaler.transform(test_data_array) 

In [37]:
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1])
        y.append(data[i+seq_length, -1])
    return np.array(X), np.array(y)

In [38]:
seq_length = 12  
X_train, y_train = create_sequences(scaled_train_data, seq_length)
X_test, y_test = create_sequences(scaled_test_data, seq_length)

In [48]:


from keras.layers import Input, LSTM, Dense
from keras.models import Model

def LSTM_MODEL(X_train, y_train, time_steps):
    input_shape = (time_steps, X_train[0].shape[1])  # (time_steps, features)
    input_layer = Input(shape=input_shape)
    lstm_layer = LSTM(units=50)(input_layer)
    output_layer = Dense(units=1)(lstm_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1)
    return model 


In [51]:
from tqdm import tqdm 
import warnings
warnings.filterwarnings("ignore")


results = list()
all_prestation = train_data.Fk_prestation_code.unique()
for i in tqdm(range(len(all_prestation))):
    prestation = all_prestation[i]
    sub_train_data = train_data[train_data.Fk_prestation_code == prestation].drop(columns = "Fk_prestation_code")

    scaler = MinMaxScaler()
    scaled_train_data = scaler.fit_transform(train_data.drop(columns="montant_HT"))
    X_train_scaled=scaled_train_data
    y_train_scaled = train_data["montant_HT"]
    scaled_test_data = scaler.transform(test_data.drop(columns="montant_HT")) 
    X_test_scaled = scaled_test_data
    y_test_scaled = test_data["montant_HT"]
    time_steps = 5
    X_train_reshaped = []
    for i in range(time_steps, len(X_train_scaled)):
        X_train_reshaped.append(X_train_scaled[i-time_steps:i]) 
    y_train_reshaped = []
    for i in range(time_steps, len(y_train_scaled)):
        y_train_reshaped.append(y_train_scaled[i-time_steps:i]) 

    #print(X_train_reshaped.shape, y_train_reshaped.shape)
    pred = LSTM_MODEL(X_train_reshaped,y_train_reshaped,time_steps=7)

  0%|          | 0/20 [00:26<?, ?it/s]


KeyboardInterrupt: 

In [52]:
X_train_reshaped[0].shape

(5, 4)

In [54]:
input_shape = (7, X_train_reshaped[0].shape[1])  # (time_steps, features)
input_layer = Input(shape=input_shape)
lstm_layer = LSTM(units=50)(input_layer)
output_layer = Dense(units=1)(lstm_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_reshaped, y_train_reshaped, epochs=4, batch_size=1, verbose=1)

KeyboardInterrupt: 

In [9]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model

input_shape = (X_train.shape[1], X_train.shape[2])

input_layer = Input(shape=input_shape)
lstm_layer = LSTM(units=50)(input_layer)
output_layer = Dense(units=1)(lstm_layer)

model = Model(inputs=input_layer, outputs=output_layer)


NameError: name 'X_train' is not defined

In [40]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [41]:
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0014
Epoch 2/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0011
Epoch 3/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0011
Epoch 4/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0010
Epoch 5/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 9.8511e-04
Epoch 6/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0011
Epoch 7/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 9.4997e-04
Epoch 8/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 9.3530e-04
Epoch 9/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 9.4795e-04
Epoch 10/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 8.6660e-04
Epoch 11/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 8.3134e-04
Epoch 12/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 8.9514e-04
Epoch 13/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 9.3138e-04
Epoch 14/100
1166/1166 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/ste

In [42]:
predicted_data = model.predict(X_test)

98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [43]:
predicted_data.sum()

187.84286

In [44]:

combined_y_test_data = np.hstack((X_test[:, -1], predicted_data.reshape(-1, 1)))
predicted_data_real = scaler.inverse_transform(combined_y_test_data)
y_test_sum = np.sum(predicted_data_real[:, -1])
print("Sum of real values of predicted_data:", y_test_sum) 

Sum of real values of predicted_data: 4080413.2743351534


Other LSTM Method

In [ ]:
selected_columns = ['date_saisie', 'montant_HT','Fk_prestation_code']
df1 = df.loc[:, selected_columns]
df1['year'] = df['date_saisie'].dt.year.astype(int)
df1['month'] = df['date_saisie'].dt.month.astype(int)
df1['day'] = df['date_saisie'].dt.day.astype(int)
selected_columns = ['year','month','day','montant_HT','Fk_prestation_code']
df1 = df1.loc[:, selected_columns]
df1

In [ ]:
df1 = df1.groupby([ 'year','month','day','Fk_prestation_code'])['montant_HT'].sum().reset_index()

In [ ]:
train_data = df1[(df1['year'] <= 2018)]
test_data = df1[df1['year'] == 2019]

In [ ]:
X_train = train_data.drop(['year','montant_HT','day','month'], axis=1)
y_train=train_data['montant_HT']
X_train_padded = train_data.drop(['montant_HT'], axis=1)
y_train_padded = train_data['montant_HT']

X_test = test_data.drop(['year','montant_HT','month','day'], axis=1)
X_test_padded = test_data.drop(['montant_HT'], axis=1)
y_test_padded = test_data['montant_HT']
y_test=test_data['montant_HT']

In [ ]:
input_type = Input(shape=(1,), name='input_type')
embedding_type = Dense(64, activation='relu')(input_type)

In [ ]:
seq_length=12
input_time = Input(shape=(seq_length, X_train_padded.shape[2]), name='input_time')
lstm_time = LSTM(units=64)(input_time)

In [ ]:
concatenated = concatenate([embedding_type, lstm_time])

In [ ]:
output = Dense(1, name='output')(concatenated)

In [ ]:
model = Model(inputs=[input_type, input_time], outputs=output)

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit({'input_type': X_train['Fk_prestation_code'], 'input_time': X_train_padded}, y_train_padded, epochs=10, batch_size=32)

In [ ]:
predictions = model.predict({'input_type': X_test['Type_prestation'], 'input_time': X_test_padded})